In [24]:
from pymilvus import MilvusClient
from pymilvus import MilvusClient, DataType
client = MilvusClient("http://localhost:19530")


In [33]:
if client.has_collection(collection_name="scenes"):
    client.drop_collection(collection_name="scenes")
    # 1. Define the schema
schema = client.create_schema(
    auto_id=False,
    enable_dynamic_field=True, # This allows 'subject' and other fields to work automatically
)

# 2. Add fields explicitly (especially the Primary Key and Vector)
schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True)
# FIX IS HERE: Explicitly use 'dim' inside the add_field call
schema.add_field(
    field_name="vector", 
    datatype=DataType.FLOAT_VECTOR, 
    dim=1024  # Use 'dim' instead of 'dimension'
)
schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=512)
client.create_collection(
    collection_name="scenes",
    # dimension=1024,  # The vectors we will use in this demo has 768 dimensions
    schema=schema
)

In [ ]:
from pymilvus import model


embedding_fn = model.dense.SentenceTransformerEmbeddingFunction(
    model_name='BAAI/bge-m3', # Tên model trên Hugging Face
    device='cuda' # Hoặc 'cuda' nếu bạn có GPU
)


Loading weights: 100%|██████████| 391/391 [00:00<00:00, 774.55it/s, Materializing param=pooler.dense.weight]                               


Dim: 1024 (1024,)
Data has 3 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'subject'])
Vector dim: 1024


In [40]:

docs = [
    "Artificial intelligence was founded as an academic discipline in 1956.",
    "Alan Turing was the first person to conduct substantial research in AI.",
    "Born in Maida Vale, London, Turing was raised in southern England.",
]

vectors = embedding_fn.encode_documents(docs)
print("Dim:", embedding_fn.dim, vectors[0].shape)  # Dim: 768 (768,)

data = [
    {"id": i, "vector": vectors[i], "text": docs[i], "subject": "history"}
    for i in range(len(vectors))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))


Dim: 1024 (1024,)
Data has 3 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'subject'])
Vector dim: 1024


Insert Data


In [41]:
res = client.insert(collection_name="scenes", data=data)

print(res)

{'insert_count': 3, 'ids': [0, 1, 2]}


In [44]:
index_params = client.prepare_index_params()
index_params.add_index(
    field_name="vector",
    index_type="HNSW",
    metric_type="COSINE",
    params={"M": 16, "efConstruction": 256}
)
client.create_index(collection_name="scenes", index_params=index_params)

Query

In [49]:
query_vectors = embedding_fn.encode_queries(["Who is Alan Turing?"])
# If you don't have the embedding function you can use a fake vector to finish the demo:
# query_vectors = [ [ random.uniform(-1, 1) for _ in range(768) ] ]
client.load_collection(collection_name="scenes")
res = client.search(
    collection_name="scenes",  # target collection
    data=query_vectors,  # query vectors
    limit=2,  # number of returned entities
    output_fields=["text", "subject"],  # specifies fields to be returned
)

print(res)

2026-02-10 16:02:58,699 [ERROR][handler]: RPC error: [search], <MilvusException: (code=65535, message=field text not exist)>, <Time:{'RPC start': '2026-02-10 16:02:58.697314', 'RPC error': '2026-02-10 16:02:58.699440'}>
Traceback:
Traceback (most recent call last):
  File "c:\Users\leduc\miniconda3\envs\md_search\Lib\site-packages\pymilvus\decorators.py", line 387, in handler
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\leduc\miniconda3\envs\md_search\Lib\site-packages\pymilvus\decorators.py", line 446, in handler
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\leduc\miniconda3\envs\md_search\Lib\site-packages\pymilvus\decorators.py", line 320, in handler
    raise e from e
  File "c:\Users\leduc\miniconda3\envs\md_search\Lib\site-packages\pymilvus\decorators.py", line 290, in handler
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\leduc\miniconda3\envs\md_search\Lib\

2026-02-10 16:02:58,702 [ERROR][handler]: RPC error: [search], <MilvusException: (code=65535, message=field text not exist)>, <Time:{'RPC start': '2026-02-10 16:02:58.697314', 'RPC error': '2026-02-10 16:02:58.701211'}>
Traceback:
Traceback (most recent call last):
  File "c:\Users\leduc\miniconda3\envs\md_search\Lib\site-packages\pymilvus\decorators.py", line 387, in handler
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\leduc\miniconda3\envs\md_search\Lib\site-packages\pymilvus\decorators.py", line 446, in handler
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\leduc\miniconda3\envs\md_search\Lib\site-packages\pymilvus\decorators.py", line 320, in handler
    raise e from e
  File "c:\Users\leduc\miniconda3\envs\md_search\Lib\site-packages\pymilvus\decorators.py", line 290, in handler
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\leduc\miniconda3\envs\md_search\Lib\

MilvusException: <MilvusException: (code=65535, message=field text not exist)>